In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
import random
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm
import os
import matplotlib.pyplot as plt
import scipy
import yfinance as yf

In [2]:
torch.__version__

'2.0.0'

### Download SP500 assets

In [3]:
with open('sectors_dict.pickle', 'rb') as f:
    sectors = pickle.load(f)

In [4]:
DATASET_DIRNAME = '../stmom/SpatioTemporalMomentum/yf_data'
PERIOD = '1day'
START_DATE = '2010-01-01'

In [ ]:
if not os.path.exists(DATASET_DIRNAME):
    os.mkdir(DATASET_DIRNAME)

In [ ]:
for key in tqdm(sectors.keys()):
    if not os.path.exists(os.path.join(DATASET_DIRNAME, key)):
        os.mkdir(os.path.join(DATASET_DIRNAME, key))
    for symbol in sectors[key]:
        data = yf.download(symbol, period=PERIOD, start=START_DATE)
        data.to_csv(os.path.join(DATASET_DIRNAME, key, '{}.csv'.format(symbol)))

### Load data of randomly chosen assets

In [5]:
SEED = 42
N_ASSETS_PER_SECTOR = 5
LENGTH_THRESHOLD = 3000

In [6]:
np.random.seed(SEED)
all_dfs = []
for current_sector in sectors.keys():
    dfs = []
    for asset in sectors[current_sector]:
        df = pd.read_csv(os.path.join(DATASET_DIRNAME, f'{current_sector}', f'{asset}.csv'))
        # use only stocks with long enough history
        if len(df) > LENGTH_THRESHOLD:
            df['Date'] = pd.to_datetime(df['Date'])

            df = df[['Date', 'Adj Close']]
            df.columns = ['Date', asset]
            dfs.append(df)

    df = dfs[0]
    for i in range(1, len(dfs)):
        df = pd.merge(df, dfs[i], left_on='Date', right_on='Date', how='inner')

    assets = list(df.columns[1:])
    used_assets = np.random.choice(assets, N_ASSETS_PER_SECTOR, replace=False)
    df = df[['Date'] + list(used_assets)]
    all_dfs.append(df)

df = all_dfs[0]
for i in range(1, len(all_dfs)):
    df = pd.merge(df, all_dfs[i], left_on='Date', right_on='Date', how='inner')
df = df.set_index(df['Date'])
print('loaded dataframe shape', df.shape)

loaded dataframe shape (3068, 56)


### Calculate log returns, center its mean to zero

In [7]:
all_assets = list(df.columns[1:])

In [8]:
# create log returns
log_returns_df = np.log(df[all_assets]) - np.log(df[all_assets].shift(1))
log_returns_df = log_returns_df.iloc[1:]
log_returns_mean = log_returns_df.mean(axis=0)
log_returns_df = log_returns_df - log_returns_mean
print('Mean of log returns dataframe', log_returns_df.mean().sum())

Mean of log returns dataframe 2.714924253240519e-18


### Split data set into train/validation/test and format it as tensors of shape (n_samples, n_timesteps, n_features)

In [9]:
validation_start = pd.Timestamp('2020-01-01')
validation_end = pd.Timestamp('2021-01-01')

In [10]:
index = np.arange(len(log_returns_df))
log_returns_df.loc[:, 'index'] = index

In [11]:
train_idxes = log_returns_df.loc[:validation_start - pd.Timedelta('1day'), 'index'].values
val_idxes = log_returns_df.loc[validation_start:validation_end, 'index'].values
test_idxes = log_returns_df.loc[validation_end + pd.Timedelta('1day'):, 'index'].values

In [12]:
log_returns_df = log_returns_df.drop(['index'], axis=1)

In [13]:
# let's try window size of 3 weeks
history_size = 21*3

In [14]:
X = np.zeros((len(log_returns_df), history_size, log_returns_df.shape[1]))
for i, col in enumerate(log_returns_df.columns):
    for j in range(history_size):
        X[:, j, i] = log_returns_df[col].shift(history_size - j - 1)

In [15]:
X_train = X[train_idxes]
X_val = X[val_idxes]
X_test = X[test_idxes]

In [16]:
X_train = X_train[history_size:]
train_idxes = train_idxes[history_size:]

In [17]:
X_train = torch.Tensor(X_train)
X_val = torch.Tensor(X_val)
X_test = torch.Tensor(X_test)

In [18]:
def _seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

# fix random seed of train dataloader for reproducibility
g = torch.Generator()
g.manual_seed(SEED)
train_loader = DataLoader(TensorDataset(X_train, X_train), shuffle=True, batch_size=128)
val_loader = DataLoader(TensorDataset(X_val, X_val), shuffle=False, batch_size=128)
test_loader = DataLoader(TensorDataset(X_test, X_test), shuffle=False, batch_size=128)

### Setup model, optimizer and learning rate 

In [19]:
import torch
from torch import nn
import numpy as np


class VRNN(torch.jit.ScriptModule):
    def __init__(self, input_dim, hidden_dim=128, device='cpu'):
        super().__init__()
        
        self.latent_dim = int(input_dim*(input_dim+1)/2)
        self.N = input_dim
        self.gru_cell = nn.GRUCell(input_dim, hidden_dim)
        self.mlp_gen = nn.Sequential(nn.Linear(hidden_dim, 2*hidden_dim),
                                     nn.LeakyReLU(),
                                     nn.Linear(2*hidden_dim, hidden_dim))
        self.mlp_gen_mu = nn.Linear(hidden_dim, self.latent_dim)
        self.mlp_gen_sigma = nn.Sequential(nn.Linear(hidden_dim, self.latent_dim), nn.Softplus())
        
        self.mlp_inf = nn.Sequential(nn.Linear(hidden_dim, 2*hidden_dim),
                                     nn.LeakyReLU(),
                                     nn.Linear(2*hidden_dim, hidden_dim))
        self.mlp_inf_mu = nn.Linear(hidden_dim, self.latent_dim)
        self.mlp_inf_sigma = nn.Sequential(nn.Linear(hidden_dim, self.latent_dim), nn.Softplus())
        
        self.softplus = nn.Softplus()
        self.hidden_dim = hidden_dim
        self.device = device
        
    
    @torch.jit.script_method
    def forward(self, r):
        h_tm1 = torch.zeros(r.shape[0], self.hidden_dim).to(self.device)
        loglike_loss = torch.tensor(0.0).to(self.device)
        kl_loss = torch.tensor(0.0).to(self.device)
        Ls = []
        
        for t in range(r.shape[1]):
            
            # calculate prior P(z_t|h_tm1)
            prior = self.mlp_gen(h_tm1)
            prior_mu = self.mlp_gen_mu(prior)
            prior_sigma = self.mlp_gen_sigma(prior)
            
            # calculate hidden state transition h_t = f(r_t, h_tm1)
            h_t = self.gru_cell(r[:, t, :], h_tm1)
            
            # calculate posterior q(z_t|h_t)
            encoded = self.mlp_inf(h_t)
            encoded_mu = self.mlp_inf_mu(encoded)
            encoded_sigma = self.mlp_inf_sigma(encoded)
            
            random_noise = torch.randn_like(prior_mu).to(self.device)
            z_t = random_noise*prior_sigma + prior_mu
            
            loglike_loss_, L_t = self._loglike_loss(z_t, r[:, t, :])
            loglike_loss += loglike_loss_
            kl_loss += self._kl_loss(encoded_mu, encoded_sigma, prior_mu, prior_sigma)
            
            h_tm1 = h_t
            Ls.append(L_t)
            
        return loglike_loss, kl_loss, Ls

    
    def _construct_choleskiy_matrix(self, z_t):
       
        L_t = torch.zeros(z_t.shape[0], self.N, self.N).to(self.device)
        idxes = torch.tril_indices(self.N, self.N)
        L_t[:, idxes[0], idxes[1]] = z_t
        
        # ensure positive definiteness
        diag_idxes = torch.arange(self.N)
        diag = L_t[:, diag_idxes, diag_idxes]
        L_t[:, diag_idxes, diag_idxes] = 0
        diag = self.softplus(diag)
        L_t[:, diag_idxes, diag_idxes] += diag
        
        return L_t, diag
    
    
    def _loglike_loss(self, z_t, r_t, eps: float=1e-9):
        
        L_t, diag = self._construct_choleskiy_matrix(z_t)
        
        # evaluate log(det(sigma_t))
        log_det = -2*torch.sum(torch.log(diag + eps), dim=1)
        
        # calculate Mahalanobis distance
        r_t = r_t.unsqueeze(2)
        r_T_t = r_t.permute(0, 2, 1)
        L_T_t = torch.transpose(L_t, 1, 2)
        
        mach_distance = r_T_t @ L_t @ L_T_t @ r_t
        
        #squeeze extra dimensions to avoid broadcasting
        mach_distance = mach_distance.squeeze(1).squeeze(1)
        
        loglike_loss = -1/2.0*(log_det + mach_distance)
        loglike_loss = torch.mean(loglike_loss)
        
        return loglike_loss, L_t
    
    
    def _kl_loss(self, mean_1, sigma_1, mean_2, sigma_2, eps: float=1e-9):

        kld_element =  (2 * torch.log(sigma_2 + eps) - 2 * torch.log(sigma_1 + eps) + 
            (sigma_1.pow(2) + (mean_1 - mean_2).pow(2)) /
            sigma_2.pow(2) - 1)
        return 0.5 * torch.sum(kld_element)

In [20]:

def _set_seed(seed) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
    
device = 'cpu'
step = 10
gamma = 0.8
early_stopping_rounds = 15
n_epochs = 100
kl_coef = 1
best_val_loss = np.PINF

# fix random seed of model's weights initialization for reproducibility
_set_seed(SEED)
model = VRNN(X_train.shape[2]).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(opt, step, gamma)

Random seed set as 42


### Train model

In [ ]:
counter = 0
train_loglike_losses = []
val_loglike_losses = []
train_kl_losses = []
val_kl_losses = []

for e in tqdm(range(n_epochs)):
    model.train()
    train_loglike_loss_epoch = 0
    train_kl_loss_epoch = 0
    for batch_x, _ in tqdm(train_loader):
        batch_x = batch_x.to(device)
        
        opt.zero_grad()
        
        loglike_loss, kl_loss, _ = model(batch_x)
        
        # we seek to maximise the multivariate normal log likelihood function, so we multiply loglike_loss by -1
        # kl_coef is used to control tradeoff between losses
        loss = -1*loglike_loss + kl_coef*kl_loss
        
        loss.backward()
        
        opt.step()
        
        train_loglike_loss_epoch += loglike_loss.item()
        train_kl_loss_epoch += kl_loss.item()
        
    train_loglike_losses.append(train_loglike_loss_epoch/len(train_loader))
    train_kl_losses.append(train_kl_loss_epoch/len(train_loader))
    
    # if lr is small enough, do not decrease it
    if scheduler.get_last_lr()[0] > 1e-5:
        scheduler.step()
        
    model.eval()
    val_loss = 0
    val_loglike_loss_epoch = 0
    val_kl_loss_epoch = 0
    with torch.no_grad():
        for batch_x, _ in tqdm(val_loader):
            batch_x = batch_x.to(device)

            opt.zero_grad()

            loglike_loss, kl_loss, _ = model(batch_x)

            loss = -1*loglike_loss + kl_coef*kl_loss
            
            val_loss += loss.item()
            val_loglike_loss_epoch += loglike_loss.item()
            val_kl_loss_epoch += kl_loss.item()
            
    val_loglike_losses.append(val_loglike_loss_epoch/len(val_loader))
    val_kl_losses.append(val_kl_loss_epoch/len(val_loader))
    
    val_loss /= len(val_loader)
    
    if np.sign(val_loss) != np.sign(best_val_loss):
        best_val_loss = val_loss
    
    # save best model's weights 
    if best_val_loss > val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), 'vrnn_zero_mean.pt')
    else:
        counter += 1
    
    # if validation loss doesn't improve for several epochs, stop training
    if counter > early_stopping_rounds:
        break
    
    print('Iter: ', e,
          'train loglike loss: ', round(train_loglike_losses[-1], 3),
          'train kl loss: ', round(train_kl_losses[-1], 3),
          'val loglike loss: ', round(val_loglike_losses[-1], 3),
          'val kl loss: ', round(val_kl_losses[-1], 3),
          'total val loss: ', round(val_loss, 3),
          'best val loss: ', round(best_val_loss, 3),
          'epochs till end: ', early_stopping_rounds - counter)

In [21]:
# load best model's weights
model.load_state_dict(torch.load('vrnn_zero_mean.pt'))

<All keys matched successfully>

In [22]:
model.eval()
print()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Log Likelihood loss')
plt.plot(train_loglike_losses, label='train', marker='o')
plt.plot(val_loglike_losses, label='validation', marker='o')
plt.ylabel('Log Likelihood')
plt.xlabel('Epochs')
plt.legend()

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('KL loss')
plt.plot(train_kl_losses, label='train', marker='o')
plt.plot(val_kl_losses, label='validation', marker='o')
plt.ylabel('KL')
plt.xlabel('Epochs')
plt.legend()

### Evaluate model performance in terms of  multivariate normal log likelihood function (higher is better)

#### (Note that inference is stochastic)

In [24]:
def loglike_loss_fn(sigma, r_t):
    r_t = r_t[:, np.newaxis]
    r_T_t = np.swapaxes(r_t, 0, 1)
    P = np.linalg.inv(sigma)
    
    mach_distance = r_T_t @ P @ r_t
    mach_distance = mach_distance[0, 0]
    
    log_det = np.log(np.linalg.det(sigma) + 1e-9)

    loss = log_det + mach_distance
    
    return loss
    

In [25]:
val_sigmas = []
val_losses = []
with torch.no_grad():
    for batch_x, _ in tqdm(val_loader):
        batch_x = batch_x.to(device)

        opt.zero_grad()

        loglike_loss, kl_loss, Ls = model(batch_x)
        
        # use last timestamp as we no longer need time dimension
        L_t = Ls[-1].numpy()
        r_t = batch_x[:, -1, :].numpy()
        L_T_t = np.transpose(L_t, axes=(0, 2, 1))
        
        for i in range(len(r_t)):
            # calculate precision matrix
            P = L_t[i] @ L_T_t[i]
            # calculate covariance matrix
            sigma = np.linalg.inv(P)
        
            val_loss = loglike_loss_fn(sigma, r_t[i])
            val_sigmas.append(sigma)
            val_losses.append(val_loss)

# ensure that calculated covariance matrix is symmetric and positive definite
for j in range(len(val_sigmas)):
    assert scipy.linalg.issymmetric(val_sigmas[j], atol=1e-8)
    assert np.all(np.linalg.eigvals(val_sigmas[j]))

val_losses = np.array(val_losses)
val_loglike_loss = -1/2.0*np.sum(val_losses)
print('VHVM validation loss', val_loglike_loss)

  0%|          | 0/2 [00:00<?, ?it/s]

VHVM validation loss 351.4297228599578


In [26]:
test_sigmas = []
test_losses = []
with torch.no_grad():
    for batch_x, _ in tqdm(test_loader):
        batch_x = batch_x.to(device)

        opt.zero_grad()

        loglike_loss, kl_loss, Ls = model(batch_x)
        
        # use last timestamp as we no longer need time dimension
        L_t = Ls[-1].numpy()
        r_t = batch_x[:, -1, :].numpy()
        L_T_t = np.transpose(L_t, axes=(0, 2, 1))
        
        for i in range(len(r_t)):
            # calculate precision matrix
            P = L_t[i] @ L_T_t[i]
            # calculate covariance matrix
            sigma = np.linalg.inv(P)
        
            test_loss = loglike_loss_fn(sigma, r_t[i])
            test_sigmas.append(sigma)
            test_losses.append(test_loss)

# ensure that calculated covariance matrix is symmetric and positive definite
for j in range(len(test_sigmas)):
    assert scipy.linalg.issymmetric(test_sigmas[j], atol=1e-8)
    assert np.all(np.linalg.eigvals(test_sigmas[j]))

test_losses = np.array(test_losses)
test_loglike_loss = -1/2.0*np.sum(test_losses)
print('VHVM test loss', test_loglike_loss)

  0%|          | 0/6 [00:00<?, ?it/s]

VHVM test loss 4878.875368932642


### Lets compare VHVM approach with Ledoit Wolf covariance estimation

In [27]:
from sklearn.covariance import LedoitWolf

In [28]:
val_sigmas_lw = []
val_losses_lw = []

idx_start = len(log_returns_df.loc[:validation_start])
idx_end = len(log_returns_df.loc[:validation_end])

for idx in tqdm(range(idx_start, idx_end)):
    df_slice = log_returns_df.iloc[:idx]
    r_t = df_slice.values[-1]

    lw = LedoitWolf()
    
    sigma_lw = lw.fit(df_slice.values).covariance_
    
    loss_lw = loglike_loss_fn(sigma_lw, r_t)
    
    val_sigmas_lw.append(sigma_lw)
    val_losses_lw.append(loss_lw)


val_losses_lw = np.array(val_losses_lw)
val_loss_lw = -1/2.0*np.sum(val_losses_lw)
print('LedoitWolf validation loss', val_loss_lw)

  0%|          | 0/253 [00:00<?, ?it/s]

LedoitWolf validation loss -13443.012589966645


In [29]:
test_sigmas_lw = []
test_losses_lw = []

idx_start = len(log_returns_df.loc[:validation_end+pd.Timedelta('1day')])
idx_end = len(log_returns_df)

for idx in tqdm(range(idx_start, idx_end)):
    df_slice = log_returns_df.iloc[:idx]
    r_t = df_slice.values[-1]

    lw = LedoitWolf()
    
    sigma_lw = lw.fit(df_slice.values).covariance_
    
    loss_lw = loglike_loss_fn(sigma_lw, r_t)
    
    test_sigmas_lw.append(sigma_lw)
    test_losses_lw.append(loss_lw)


test_losses_lw = np.array(test_losses_lw)
test_loss_lw = -1/2.0*np.sum(test_losses_lw)
print('LedoitWolf test loss', test_loss_lw)

  0%|          | 0/671 [00:00<?, ?it/s]

LedoitWolf test loss -13256.787454213252
